In [1]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os

In [2]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)



In [3]:
# Get only the WRR items
wrr_df = mss_df.loc[mss_df['UniqueID'].str.contains("WRR")].copy()

In [4]:
# Make rough draft of the client table
client_df = wrr_df[['UniqueID',
                    'Tract',
                    'CMP',
                    'Stand',
                    'Acres',
                    'Age',
                    'County',
                    'State',
                    'Stand Type',
                    'Origin',
                    'Scheduled',
                    'PreBA',
                    'PreTPA',
                    'PreGTA',
                    'PlnBA',
                    'PlnTPA',
                    'PlnGTA',
                    'Client GT',
                    'Future GT',
                    'Reporting Period']].copy()

# fix up Reporting Period column
client_df['Reporting Period'] = client_df['Reporting Period'].fillna(0)
client_df.loc[client_df['Reporting Period'] == "", 'Reporting Period'] = 0
s = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'].apply(lambda x: x.strip()[-1])
client_df['trial'] = client_df.index.map(s)
client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'] = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'trial']
client_df['Reporting Period'] = pd.to_numeric(client_df['Reporting Period'], downcast='integer')
# Fix up age column
client_df['Age'] = pd.to_numeric(client_df['Age'], errors='coerce', downcast='integer').fillna(0).astype(int)
# fix up future gt column
client_df.loc[pd.to_numeric(client_df['Future GT'], errors='coerce').isnull(), 'Future GT'] = 0
# fix up Tract and Stand columns
client_df['Tract'] = client_df['Tract'].astype(str)
client_df['Stand'] = client_df['Stand'].astype(str)
client_df['Tract'] = client_df['Tract'].apply(lambda x: x.split('.')[0])
# fix up client gt column
client_df['Client GT'] = pd.to_numeric(client_df['Client GT'], errors="coerce")
client_df['Client GT'] = client_df['Client GT'].fillna(0)

# fix up harvest instake columns
cols = ['PreBA', 'PreTPA', 'PreGTA', 'PlnBA', 'PlnTPA', 'PlnGTA']

for x in cols:
    client_df[x] = pd.to_numeric(client_df[x], errors='coerce', downcast='integer').fillna(-99).astype(int)

client_df.drop(columns=['trial'],inplace=True)


# Normalize species

species = ["Slash Pine", "Loblolly Pine", 'Mixed Natural Pine', 'Hardwood Mix', 'Pine Straw', "Other"]
species_dict = {'Species': species}
species_df = pd.DataFrame(species_dict)
species_df['SpeciesUID'] = species_df.index

def species_map(x):
    x = x.lower()
    if "slash" in x:
        return 0
    elif "loblol" in x:
        return 1
    elif "mixed natural" in x:
        return 2
    elif "h" in x:
        return 3
    elif "straw" in x:
        return 4
    
    elif "pine" in x:
        return 1
    else:
        return 5
    
client_df['Stand Type'] = client_df['Stand Type'].apply(lambda x: species_map(x))

# Normalize Origin

origins = ['Planted', 'Natural', 'Natural Mix', 'Other']
origin_dict = {'Origin': origins}
origin_df = pd.DataFrame(origin_dict)
origin_df['OriginUID'] = origin_df.index

def origin_map(x):
    x = x.lower()
    if "plant" in x:
        return 0
    elif "natural mix" in x:
        return 2
    elif "nat" in x:
        return 1
    else:
        return 3
client_df['Origin'] = client_df['Origin'].apply(lambda x: origin_map(x))



In [5]:
"""
client_df.columns = [x.lower() for x in client_df.columns]
client_df.columns = [x.replace(' ', '_') for x in client_df.columns]
client_df['id'] = client_df['uniqueid']
client_df.drop(columns=['uniqueid'],inplace=True)
client_df = client_df[['id', 'tract', 'cmp', 'stand', 'acres', 'age', 'county', 'state', 'stand_type', 'origin', 'scheduled', 'preba', 'pretpa', 'pregta',
                       'plnba', 'plntpa', 'plngta', 'client_gt', 'future_gt', 'reporting_period']]
"""

"\nclient_df.columns = [x.lower() for x in client_df.columns]\nclient_df.columns = [x.replace(' ', '_') for x in client_df.columns]\nclient_df['id'] = client_df['uniqueid']\nclient_df.drop(columns=['uniqueid'],inplace=True)\nclient_df = client_df[['id', 'tract', 'cmp', 'stand', 'acres', 'age', 'county', 'state', 'stand_type', 'origin', 'scheduled', 'preba', 'pretpa', 'pregta',\n                       'plnba', 'plntpa', 'plngta', 'client_gt', 'future_gt', 'reporting_period']]\n"

In [6]:
ga_intake_df = wrr_df[['IntakeID',
                       'FolderID',
                       'UniqueID',
                       'GA_ACRES',
                       'GA GT', # this is going to be GA total isn't it? Or does it stay as GA estimate of current GT (total minus what client says is future)
                       'Total GT', # Does this need included? At the moment we do funny stuff with total gt and subtracting future gt to get GA current rp (above)
                       'Notes'
                       ]].copy()

# Do I want to include a Ticketid column here or use a crossreference table?
# Should reporting period be in here?

In [7]:
status_df = wrr_df[['UniqueID',
                    'Guidance',
                    'Expiration',
                    'GuidanceProvider',
                    'Status',
                    'Completed Date'
                    ]].copy()

In [8]:
def intake_breaker(lst):
    intake_lst = []
    activity_lst = []
    for ele in lst:
        intake_lst.append(ele[:9])
        activity_lst.append(ele[9:11])
        intake_st = set(intake_lst)
        activity_st = set(activity_lst)
        
    return intake_lst, activity_lst, intake_st, activity_st


def ticketid(lst):
    """This function creates a ticket id based off a list of unique ids

    Args:
        lst (list): a list of unique ids that have the same Tract_Cmp_Stand from MSS
    """
    if len(lst) == 0:
        return "ERROR, Where is your uniqueid?"
    elif len(lst) < 2:
        return lst[0]
    elif len(lst) > 1:
        # Check to see if there are multiple intakes
        temp_lst, activity_lst, temp_st, activity_st = intake_breaker(lst)
        temp_lst.sort()
        activity_lst.sort()
        if len(temp_st) > 1:
            return f"{temp_lst[0]}XXXX"
        # If there are not multiple intakes. Check to see if there are multiple activities in the lst (Intake portion should be same for all)
        elif len(activity_st) > 1:
           return f"{temp_lst[0]}XXXX"
       # If there is only a single activity type
        elif len(activity_st) == 1:
            return f"{temp_lst[0]}{activity_lst[0]}XX"
        else:
            return "Failed within len(lst) > 1"
        
    else:
        return "Failed at length of lst" 

In [9]:
wrr_inprog = wrr_df[wrr_df['Status'].isin(['OPEN', 'IN PROCESS', 'POST', 'PENDING'])].copy()
wrr_inprog['Tract'] = wrr_inprog['Tract'].astype(str).apply(lambda x: x.split('.')[0])
wrr_inprog['CMP'] = wrr_inprog['CMP'].astype(str).apply(lambda x: x.split('.')[0])
wrr_inprog['Stand'] = wrr_inprog['Stand'].astype(str).apply(lambda x: x.split('.')[0])
wrr_inprog['TCS'] = wrr_inprog['Tract'] + "_" + wrr_inprog['CMP'] + "_" + wrr_inprog['Stand']

# After talking with Rob, we need to make sure that we separate out the GA and the ARB Guidance
grouped_activities = wrr_inprog.groupby(['TCS', 'GuidanceProvider'])['UniqueID'].apply(list).reset_index()

trial_df = grouped_activities.copy()
trial_df['BBID'] = trial_df['UniqueID'].apply(lambda x: ticketid(x))

prev_combined = wrr_inprog.groupby('TicketID')['TCS'].apply(set).reset_index()

tcs = []
for val in prev_combined['TCS'].loc[prev_combined['TCS'].str.len()> 1].values:
    for item in val:
        tcs.append(item)

# looking for TCS combinations in previous ticketids        
ticket_id_dict = {}
for i, row in prev_combined.loc[prev_combined['TCS'].str.len()> 1].iterrows():
    # print(row['TicketID']) # ticket id
    value_list = []
    for value in row['TCS']:
        value_list.append(value)
    ticket_id_dict[row['TicketID']] = value_list
if len(ticket_id_dict) > 0:
    for k, v in ticket_id_dict.items():
        trial_df.loc[trial_df['TCS'].isin(v), 'otherid'] = k
else:
    trial_df['otherid'] = None
    

def final_id(row):
    if row['otherid'] != None:
        return row['otherid']
    else:
        return row['BBID']
    
trial_df['finalid'] = trial_df[['BBID', 'otherid']].apply(final_id, axis=1)

def break_list(x):
    trial_list = []
    for item in x:
        for iter in item:
            trial_list.append(iter)
            
    return trial_list

new_ticket_df = trial_df.groupby('finalid').agg({'TCS': lambda x: list(x),
                                 'UniqueID': lambda x: break_list(x)}).reset_index()

client_for_ticket = client_df.copy()
for id in new_ticket_df['finalid'].unique().tolist():
    client_for_ticket.loc[client_for_ticket['UniqueID'].isin(new_ticket_df.loc[new_ticket_df['finalid'] == id, 'UniqueID'].values[0]), 'TicketID'] = id

new_tcs_intake = client_for_ticket.groupby(['TicketID',
                                            'UniqueID'])['Client GT'].sum().reset_index()

old_ticket_ids = dict(zip(wrr_df['UniqueID'], wrr_df['TicketID']))
new_tcs_intake['old_ticketids'] = new_tcs_intake['UniqueID'].map(old_ticket_ids)
new_tcs_intake.loc[~new_tcs_intake['TicketID'].str.contains("WRR"), 'TicketID'] = new_tcs_intake['old_ticketids']


ticketid_table = new_tcs_intake[['TicketID', 'UniqueID']]

Getting the mill tickets

In [71]:
mill_loc = r"P:\Mill Tickets"
file_pattern = "WRR_RP*_Compiled.xlsm"
big_df = pd.DataFrame()

mill_name = []
for root, dirs, files in os.walk(mill_loc):
    for file in files:
        if "WRR_RP" in file and file.endswith("Compiled.xlsm"):
            if "$" in file:
                pass
            else:
                print(root)
                print(file)
                df = pd.read_excel(os.path.join(root, file), sheet_name='RP_Project')
                big_df = pd.concat([big_df, df])
                

# big_df.loc[big_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = big_df.loc[big_df['Tons (U.S)'].isnull(), 'TonsUS']
# big_df['Cull Tons'].fillna(big_df['CullTons'], inplace=True)
# big_df['Accepted Tons'].fillna(big_df['AcceptedTons'], inplace=True)
# big_df['Reporting Period'].fillna(big_df['ReportingPeriod'],inplace=True)

def rep_fix(x):
    if "RP" in str(x):
        return x[-1]
    else:
        return x
    
big_df['Reporting Period'] = big_df['Reporting Period'].apply(lambda x: rep_fix(x))
big_df['Reporting Period'] = pd.to_numeric(big_df['Reporting Period'], errors='coerce')

big_df['Date'] = pd.to_datetime(big_df['Date'])

mill_df = big_df[['Date',
                  'Ticket',
                  'Mill',
                  'Product',
                  'Tons (U.S)',
                  'Cull Tons',
                  'Accepted Tons',
                  'Source',
                  'Notes',
                  'Reporting Period',
                  'TicketID',
                  'Week']].copy()

mill_df = mill_df.drop_duplicates()

P:\Mill Tickets\ARCHIVE\WRR_RP5
WRR_RP5_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WRR_RP6
WRR_RP6_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WRR_RP7
WRR_RP7_Compiled.xlsm
P:\Mill Tickets\WRR_RP8
WRR_RP8_Compiled.xlsm


In [72]:
mill_df.head()

,Date,Ticket,Mill,Product,Tons (U.S),Cull Tons,Accepted Tons,Source,Notes,Reporting Period,TicketID,Week
0,2021-07-06,875937,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,23.27,0.0,23.27,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN
1,2021-07-06,875841,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,23.86,0.0,23.86,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN
2,2021-07-06,875787,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,23.99,0.0,23.99,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN
3,2021-07-06,876023,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,28.16,0.0,28.16,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN
4,2021-07-06,876026,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,28.04,0.0,28.04,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN


In [73]:
# Getting the old Harvest TicketIDs
harvested_tons_oldTID = mill_df[mill_df['Reporting Period'] == 7].groupby("TicketID")['Tons (U.S)'].sum().reset_index()

In [74]:
harvested_tons_oldTID['Tons (U.S)'].sum()

63343.458

Let's get new and old TIDs that are currently in process

In [75]:
# Getting harvest ids that were in RP 7 and are still "Active" (Potentially need updating due to harvest)
harvest_activities = wrr_df[(wrr_df['Reporting Period'] == 7) & (wrr_df['Status'].isin(['IN PROCESS', "POST"]))]['UniqueID'].unique().tolist()
harvest_old_ticketid = wrr_df[(wrr_df['Reporting Period'] == 7) & (wrr_df['Status'].isin(['IN PROCESS', "POST"]))]['TicketID'].unique().tolist()

In [76]:
# Let's see how many tons are under old TIDs of activities in question
harvested_tons_oldTID[harvested_tons_oldTID['TicketID'].isin(harvest_old_ticketid)]['Tons (U.S)'].sum()

15073.17

In [77]:
# Let's get the new TIDs based on these harvest activities
new_ticketIDs = ticketid_table[ticketid_table['UniqueID'].isin(harvest_activities)]['TicketID'].unique().tolist()


In [78]:
# Let's see what is in old_ticketid that is not in new_ticketIDs
[item for item in harvest_old_ticketid if item not in new_ticketIDs]

['WRR240101CCXX',
 'WRR231219BN05',
 'WRR231219BN04',
 'WRR231219BN03',
 'WRR231219BN02',
 'WRR231219BN01',
 'WRR231107CC02',
 'WRR231107TT01']

In [79]:
for id in harvest_old_ticketid:
    old_unique = wrr_df[wrr_df['TicketID'] == id]['UniqueID'].unique().tolist()
    new_unique = ticketid_table[ticketid_table['TicketID'] == id]['UniqueID'].unique().tolist()
    inold_notinew = [item for item in old_unique if item not in new_unique]
    if len(inold_notinew) > 0:
        print(id, old_unique)

WRR240101CCXX ['WRR240101CC03', 'WRR240101CC02', 'WRR240101CC01']
WRR231219BN05 ['WRR231219BN05']
WRR231219BN04 ['WRR231219BN04']
WRR231219BN03 ['WRR231219BN03']
WRR231219BN02 ['WRR231219BN02']
WRR231219BN01 ['WRR231219BN01']
WRR231107CC02 ['WRR231107CC02']
WRR231107TT01 ['WRR231107TT01']


Ok, so some old ticketids don't line up with the new ticket ids

Let's update the mill compiled file and replace the ticket ids

In [80]:
mill_df[mill_df['TicketID'].str.contains("WRR231107CC02")]['TicketID'].unique().tolist()

['WRR231107CC02']

In [81]:
# Game plan is to get the old uniqueid -> ticketid connection and then find the new ticketid.
# Then replace all ticketids in the mill df to the new using a map

wrr_df_uid_tid = wrr_df[['UniqueID', 'TicketID']].copy()
wrr_df_uid_tid = pd.merge(wrr_df_uid_tid, ticketid_table, on='UniqueID', how="left")

In [82]:
wrr_df_uid_tid[wrr_df_uid_tid['TicketID_y'].str.contains('X')]
wrr_df_tids = wrr_df_uid_tid[['TicketID_x', 'TicketID_y']].copy()

In [83]:
wrr_df_tids.drop_duplicates(inplace=True)
wrr_tid_mapper = dict(zip(wrr_df_tids['TicketID_x'], wrr_df_tids['TicketID_y']))

In [84]:
mill_df['new_ticketid'] = mill_df['TicketID'].map(wrr_tid_mapper)

In [85]:
new_ticket_harvested = mill_df.groupby('new_ticketid')['Tons (U.S)'].sum().reset_index()
new_ticket_harvested['Tons (U.S)'].sum()

257034.38800000004

In [86]:
mill_df['Tons (U.S)'].sum()

257034.388

In [87]:
mill_df[mill_df['new_ticketid'].isna()]

,Date,Ticket,Mill,Product,Tons (U.S),Cull Tons,Accepted Tons,Source,Notes,Reporting Period,TicketID,Week,new_ticketid


In [88]:
# There was an issue where WRR230220HD01 was mislabeled WRR230220CC01 in the RP7 update compiled. I fixed that at the source on 2024.11.04

In [91]:
harvested_tons_oldTID.to_excel("harvested_tons_compare_old.xlsx", sheet_name='oldTID')
new_ticket_harvested.to_excel("harvested_tons_compare_new.xlsx", sheet_name='newTID')

In [20]:
trial_expected = pd.merge(ticketid_table, wrr_df[['UniqueID', 'Current RP Expected GT', 'Status', 'Future GT']], on='UniqueID', how='left')
trial_expected = pd.merge(trial_expected, client_df[['UniqueID', 'Reporting Period']], on='UniqueID', how='left')
wrr_rp7_expected = trial_expected[(trial_expected['Reporting Period'] == 7) & trial_expected['Status'].isin(['IN PROCESS', 'OPEN', 'POST'])].groupby('TicketID')['Current RP Expected GT'].sum().reset_index()
wrr_rp7_expected = pd.merge(wrr_rp7_expected, harvested_tons_oldTID, on='TicketID', how='left')
wrr_rp7_expected['Tons (U.S)'] = pd.to_numeric(wrr_rp7_expected['Tons (U.S)'], errors='coerce', downcast='float')
wrr_rp7_expected['Current RP Expected GT'] = pd.to_numeric(wrr_rp7_expected['Current RP Expected GT'], errors='coerce', downcast='float')
wrr_rp7_expected['Tons (U.S)'].fillna(0, inplace=True)
wrr_rp7_expected['Current RP Expected GT'].fillna(0, inplace=True)
wrr_rp7_expected['Remaining'] = wrr_rp7_expected['Current RP Expected GT'] - wrr_rp7_expected['Tons (U.S)']

In [21]:
wrr_rp7_expected[wrr_rp7_expected['Remaining'] > 0]

,TicketID,Current RP Expected GT,Tons (U.S),Remaining
0,WRR230829CC04,252.679993,0.000000,252.679993
1,WRR230829CCXX,8077.149902,5586.919922,2490.229980
4,WRR231107CC01,891.099976,700.049988,191.049988
5,WRR231107CC03,903.630005,0.000000,903.630005
6,WRR231107TH01,282.130005,215.100006,67.029999
7,WRR231107XXXX,10987.530273,0.000000,10987.530273
9,WRR240101TH01,541.820007,0.000000,541.820007
10,WRR240101TH02,888.250000,0.000000,888.250000
11,WRR240101TT01,999.900024,0.000000,999.900024
12,WRR240101XXXX,12298.480469,0.000000,12298.480469


In [22]:
# wrr_rp7_expected.to_excel("wrr_rp7_accouting.xlsx")

In [23]:
wrr_rp8_expected = trial_expected[(trial_expected['Reporting Period'] == 8) & (trial_expected['Status'].isin(['OPEN', 'IN PROCESS', 'PENDING', 'POST']))].groupby('TicketID')['Current RP Expected GT'].sum().reset_index()
wrr_rp8_expected_1 = trial_expected[(trial_expected['Reporting Period'] == 7) & (trial_expected['Status'].isin(['OPEN', 'IN PROCESS', 'PENDING', 'POST']))].groupby('TicketID')['Future GT'].sum().reset_index()

In [24]:
wrr_rp8_expected = pd.merge(wrr_rp8_expected, wrr_rp8_expected_1, on='TicketID', how='outer')

In [25]:
wrr_rp8_expected['Current RP Expected GT'] = pd.to_numeric(wrr_rp8_expected['Current RP Expected GT'], errors='coerce', downcast='float')
wrr_rp8_expected['Future GT'] = pd.to_numeric(wrr_rp8_expected['Future GT'], errors='coerce', downcast='float')
wrr_rp8_expected.fillna(0, inplace=True)
wrr_rp8_expected['RP8_Expected'] = wrr_rp8_expected['Current RP Expected GT'] + wrr_rp8_expected['Future GT']

In [26]:
wrr_rp8_trial = pd.merge(wrr_rp8_expected, wrr_rp7_expected, on='TicketID', how='left')

In [27]:
wrr_rp8_trial
wrr_rp8_trial.fillna(0, inplace=True)
wrr_rp8_trial['final_expected'] = wrr_rp8_trial['RP8_Expected'] + wrr_rp8_trial['Remaining']

Nice, this adds up with Rob's file. Lets get the info needed to fill out the Shared Space table

In [28]:
wrr_ticketid_volume = wrr_rp8_trial[['TicketID', 'final_expected']]

In [29]:
ticket_mapper = dict(zip(ticketid_table['UniqueID'], ticketid_table['TicketID']))

In [30]:
wrr_df['new_ticket_id'] = wrr_df['UniqueID'].map(ticket_mapper)

In [ ]:
# wrr_rp8_trial.to_excel('wrr_rp8_investigate_3.xlsx')

In [40]:
ticketid_table.loc[ticketid_table['TicketID'] == 'WRR231107XXXX']

,TicketID,UniqueID
11,WRR231107XXXX,WRR231107CC02
12,WRR231107XXXX,WRR231107TT01


In [92]:
# Ok. Let's create the client facing table and get it up on the drive


In [96]:
client_df.head()

,UniqueID,Tract,CMP,Stand,Acres,Age,County,State,Stand Type,Origin,Scheduled,PreBA,PreTPA,PreGTA,PlnBA,PlnTPA,PlnGTA,Client GT,Future GT,Reporting Period
43,WRR240919TT04,Mayhaw,122-1,10,19.14,29,EARLY,GA,1,0,,130,250,60,70,180,30,574.20,0.0,8
44,WRR240919TT03,Mayhaw,122-1,9,14.55,42,EARLY,GA,0,0,,115,175,60,70,95,35,363.75,0.0,8
45,WRR240919TT02,Mayhaw,122-1,9,41.84,42,EARLY,GA,0,0,,115,175,60,70,95,35,1046.00,0.0,8
46,WRR240919TT01,Mayhaw,122-1,8,16.27,44,EARLY,GA,0,0,,130,190,70,70,95,35,569.45,0.0,8
47,WRR240919TH01,Mayhaw,122-1,6,15.32,28,Early,GA,1,0,,125,350,67,70,200,40,413.64,0.0,8


In [95]:
ga_intake_df.head()

,IntakeID,FolderID,UniqueID,GA_ACRES,GA GT,Total GT,Notes
43,WRR240919,WRR240919,WRR240919TT04,19.14,574.16,574.2,A bunch of finnagling had to be done to get th...
44,WRR240919,WRR240919,WRR240919TT03,14.56,363.9,363.9,A bunch of finnagling had to be done to get th...
45,WRR240919,WRR240919,WRR240919TT02,41.85,1046.19,1046.19,A bunch of finnagling had to be done to get th...
46,WRR240919,WRR240919,WRR240919TT01,16.27,569.54,569.54,A bunch of finnagling had to be done to get th...
47,WRR240919,WRR240919,WRR240919TH01,15.32,413.68,413.68,A bunch of finnagling had to be done to get th...


In [99]:
mill_df.head() # make sure to use the correct ticketid long term

,Date,Ticket,Mill,Product,Tons (U.S),Cull Tons,Accepted Tons,Source,Notes,Reporting Period,TicketID,Week,new_ticketid
0,2021-07-06,875937,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,23.27,0.0,23.27,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN,WRR210317XXXX
1,2021-07-06,875841,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,23.86,0.0,23.86,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN,WRR210317XXXX
2,2021-07-06,875787,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,23.99,0.0,23.99,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN,WRR210317XXXX
3,2021-07-06,876023,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,28.16,0.0,28.16,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN,WRR210317XXXX
4,2021-07-06,876026,Georgia-Pacific Corporation - Cedar Springs Pa...,PPW,28.04,0.0,28.04,Mayhaw_Week 10.pdf,NaN,5.0,WRR210317XXXX,NaN,WRR210317XXXX


In [100]:
ticketid_table

,TicketID,UniqueID
0,WRR230707MW02,WRR230707MW02
1,WRR230707MW03,WRR230707MW03
2,WRR230829CC04,WRR230829CC04
3,WRR230829CCXX,WRR230829CC01
4,WRR230829CCXX,WRR230829CC02
...,...,...
267,WRR240318BN02,WRR240318BN02
268,WRR240325BN01,WRR240325BN01
269,WRR240712CC01,WRR240712CC01
270,WRR240712CC02,WRR240712CC02


In [101]:
new_tcs_intake

,TicketID,UniqueID,Client GT,old_ticketids
0,WRR230707MW02,WRR230707MW02,0.000000,WRR230707MW02
1,WRR230707MW03,WRR230707MW03,0.000000,WRR230707MW03
2,WRR230829CC04,WRR230829CC04,252.500000,WRR230829CC04
3,WRR230829CCXX,WRR230829CC01,3148.239990,WRR230829CCXX
4,WRR230829CCXX,WRR230829CC02,1755.079956,WRR230829CCXX
...,...,...,...,...
267,WRR240318BN02,WRR240318BN02,0.000000,WRR240318BN02
268,WRR240325BN01,WRR240325BN01,0.000000,WRR240325BN01
269,WRR240712CC01,WRR240712CC01,1141.150000,WRR240712CC01
270,WRR240712CC02,WRR240712CC02,1467.250000,WRR240712CC02


In [104]:
wrr_inprog[['UniqueID', 'TCS']]

,UniqueID,TCS
43,WRR240919TT04,Mayhaw_122-1_10
44,WRR240919TT03,Mayhaw_122-1_9
45,WRR240919TT02,Mayhaw_122-1_9
46,WRR240919TT01,Mayhaw_122-1_8
47,WRR240919TH01,Mayhaw_122-1_6
...,...,...
145,WRR230829CC03,Mayhaw_121-1_2
146,WRR230829CC02,Mayhaw_121-1_2
147,WRR230829CC01,Mayhaw_121-1_2
159,WRR230707MW03,Telogia_16_1


In [105]:
status_df

,UniqueID,Guidance,Expiration,GuidanceProvider,Status,Completed Date
43,WRR240919TT04,2024-09-27,,GA,OPEN,
44,WRR240919TT03,2024-09-27,,GA,OPEN,
45,WRR240919TT02,2024-09-27,,GA,OPEN,
46,WRR240919TT01,2024-09-27,,GA,OPEN,
47,WRR240919TH01,2024-09-27,,GA,OPEN,
...,...,...,...,...,...,...
334,WRR171114TH06,2017-11-14,,,EXPIRED,
335,WRR171114TH05,2017-11-14,,,COMPLETED,
336,WRR171114TH04,2017-11-14,,,COMPLETED,
337,WRR171114TH03,2017-11-14,,,COMPLETED,


In [106]:
client_table_inprog = status_df[status_df['Status'].isin(['OPEN', 'IN PROCESS', 'POST'])].copy()
client_table_inprog = pd.merge(client_table_inprog, wrr_inprog, on='UniqueID', how='left')
client_table_inprog = pd.merge(client_table_inprog, ticketid_table, on='UniqueID', how='left')

In [112]:
client_table_inprog.columns

Index(['UniqueID', 'Guidance_x', 'Expiration_x', 'GuidanceProvider_x',
       'Status_x', 'Completed Date_x', 'row_id', 'parent_id', 'IntakeID',
       'FolderID', 'TicketID_x', 'Tract', 'CMP', 'Stand', 'Acres', 'Age',
       'GA_ACRES', 'County', 'State', 'Plot Numbers', 'Status_y', 'Guidance_y',
       'Expiration_y', 'GuidanceProvider_y', 'Scheduled', 'Stand Type',
       'Origin', 'PreBA', 'PreTPA', 'PreGTA', 'PlnBA', 'PlnTPA', 'PlnGTA',
       'Client GT', 'GA GT', 'Current RP Expected GT', 'Future GT', 'Total GT',
       'Timeframe', 'Reporting Period', 'Activity Implemented',
       'Applied GA_ACRES', 'PstBA', 'PstTPA', 'PstGTA', 'RP Completed',
       'Notes', 'Completed Date_y', 'Modified', 'Modified By', 'TCS',
       'TicketID_y'],
      dtype='object')

In [117]:
client_table_inprog = client_table_inprog[['UniqueID', 'TCS', 'Status_x', 'Guidance_x', 'TicketID_y']]

In [123]:
expected_aligned_with_rob = pd.read_excel("wrr_rp8_investigate_3.xlsx")
expected_aligned_with_rob = expected_aligned_with_rob[['TicketID', 'Final Values']]
expected_aligned_with_rob['Final Values'] = pd.to_numeric(expected_aligned_with_rob['Final Values'])

In [124]:
client_table_inprog = pd.merge(client_table_inprog, expected_aligned_with_rob, left_on='TicketID_y', right_on='TicketID', how='left')

In [128]:
client_table_inprog.to_excel("looksie.xlsx")

In [135]:
client_final_df = client_table_inprog.groupby('TicketID_x').agg({'TCS': ", ".join,
                                               'Status_x': ", ".join,
                                               'Guidance_x': ", ".join,
                                               'Final Values_y': 'first'}).reset_index()

In [136]:
client_final_df['Final Values_y'].sum()

84259.90051269531